In [1]:
import data
from utils import distances

symale = [ 2 if i > 7 else 1 for i in data.SYAMALE_MEENAKSHI["melody"]]
orig = [ i   for i in data.AH_VOUS_ORIGINAL["melody"]][1:-1]
orig = [ 2 if i > 7 else 1 for i in orig]
moz_thm = []


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(symale, orig, len(symale), len(orig)),
                                                 distances.editDist(symale, orig, len(symale), len(orig)),
                                                 distances.etcDist(symale, orig),
                                                 distances.hammingDist(symale, orig, len(symale), len(orig))))

LCS: 11, ED: 27, dETC: 10.0, HD: 0.4807692307692308


In [9]:
import ETC

def get_etc_modified_data(dict):
    ret = []
    for ind in range(len(dict["melody"])):
        if dict["melody"][ind]!= 99999:
            if dict["melody"][ind] > 7:
                ret.append(2)
            else:
                ret.append(1)
    return ret

m1 = get_etc_modified_data(data.AH_VOUS_ORIGINAL)
m2 = get_etc_modified_data(data.SYAMALE_MEENAKSHI)
m3 = get_etc_modified_data(data.MOZART_THEME)
m4 = get_etc_modified_data(data.MOZART_VARIATION1)

etc1  = ETC.compute_1D(get_etc_modified_data(data.SYAMALE_MEENAKSHI), order=2, verbose=False, truncate=True)["ETC1D"]
etc2  = ETC.compute_1D(get_etc_modified_data(data.AH_VOUS_ORIGINAL), order=2, verbose=False, truncate=True)["ETC1D"]
etc3  = ETC.compute_1D(get_etc_modified_data(data.MOZART_THEME), order=2, verbose=False, truncate=True)["ETC1D"]
etc4  = ETC.compute_1D(get_etc_modified_data(data.MOZART_VARIATION1), order=2, verbose=False, truncate=True)["ETC1D"]


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(symale, orig, len(symale), len(orig)),
                                                 distances.editDist(symale, orig, len(symale), len(orig)),
                                                 distances.etcDist(symale, orig),
                                                 distances.hammingDist(symale, orig, len(symale), len(orig))))





etc1, etc2, etc3, etc4

(18, 21, 21, 21)